# Libraries

Always execute this cell at the beginning

In [1]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:4 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Ign:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:8 https://r2u.stat.illinois.edu/ubuntu jammy Release [5,713 B]
Get:9 https://r2u.stat.illinois.edu/ubuntu jammy Release.gpg [793 B]
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:13 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [2,648 kB]
Get:14 http://archi

# Create a DataFrame

In [2]:
data = [
    (1001, "Ana Perez", 20, "Engineering", 8.5),
    (1002, "Luis Gomez", 22, "Engineering", 9.0),
    (1003, "Maria Lopez", 21, "Engineering", 7.8),
    (1004, "Javier Sanchez", 23, "Architecture", 6.5),
    (1005, "Carmen Rodriguez", 20, "Architecture", 8.2)
]

schema = T.StructType([
    T.StructField("id", T.IntegerType(), True),
    T.StructField("name", T.StringType(), True),
    T.StructField("age", T.IntegerType(), True),
    T.StructField("degree", T.StringType(), True),
    T.StructField("grade", T.FloatType(), True)
])

df = spark.createDataFrame(data, schema)

df.show()

+----+----------------+---+------------+-----+
|  id|            name|age|      degree|grade|
+----+----------------+---+------------+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|
|1002|      Luis Gomez| 22| Engineering|  9.0|
|1003|     Maria Lopez| 21| Engineering|  7.8|
|1004|  Javier Sanchez| 23|Architecture|  6.5|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|
+----+----------------+---+------------+-----+



# Select columns

**select** operación de transformación, así que es perezosa (**lazy**) y no se ejecuta hasta que no es estrictamente necesario.

In [3]:
df.select('name', 'degree')

DataFrame[name: string, degree: string]

Para visualizar la información, debemos indicar una **acción** como **show** para que se ejecute la operación de seleccionar columnas.

In [4]:
df.select('name', 'degree').show()

+----------------+------------+
|            name|      degree|
+----------------+------------+
|       Ana Perez| Engineering|
|      Luis Gomez| Engineering|
|     Maria Lopez| Engineering|
|  Javier Sanchez|Architecture|
|Carmen Rodriguez|Architecture|
+----------------+------------+



**F.col** es otra forma de hacer referencia a las columnas de una tabla. No es la forma más concisa de hacer un **select**, pero para otras operaciones sí es necesario utilizarlo.

In [5]:
df.select(F.col('name'), F.col('degree')).show()

+----------------+------------+
|            name|      degree|
+----------------+------------+
|       Ana Perez| Engineering|
|      Luis Gomez| Engineering|
|     Maria Lopez| Engineering|
|  Javier Sanchez|Architecture|
|Carmen Rodriguez|Architecture|
+----------------+------------+



La forma más concisa de seleccionar columnas es con los corchetes, pero dentro del estilo de programación de PySpark, a veces es más legible usar **select**. Queda a criterio del desarrollador cuál usar en cada momento.

In [6]:
df['name', 'degree'].show()

+----------------+------------+
|            name|      degree|
+----------------+------------+
|       Ana Perez| Engineering|
|      Luis Gomez| Engineering|
|     Maria Lopez| Engineering|
|  Javier Sanchez|Architecture|
|Carmen Rodriguez|Architecture|
+----------------+------------+



# Filter rows

**filter** también es una **transformación**.

In [7]:
df.filter(F.col('degree') == 'Engineering')

DataFrame[id: int, name: string, age: int, degree: string, grade: float]

In [8]:
df.filter(F.col('degree') == 'Engineering').show()

+----+-----------+---+-----------+-----+
|  id|       name|age|     degree|grade|
+----+-----------+---+-----------+-----+
|1001|  Ana Perez| 20|Engineering|  8.5|
|1002| Luis Gomez| 22|Engineering|  9.0|
|1003|Maria Lopez| 21|Engineering|  7.8|
+----+-----------+---+-----------+-----+



Se pueden concatenar varias condiciones indicándolas entre **paréntesis**.

In [9]:
df.filter((F.col('degree') == 'Engineering') & (F.col('grade') > 8)).show()

+----+----------+---+-----------+-----+
|  id|      name|age|     degree|grade|
+----+----------+---+-----------+-----+
|1001| Ana Perez| 20|Engineering|  8.5|
|1002|Luis Gomez| 22|Engineering|  9.0|
+----+----------+---+-----------+-----+



**Recuerda**: para que una operación aplique sobre el DataFrame que estamos usando, debemos asignar el resultado después de la operación.
```
df = df.filter(F.col('degree') == 'Engineering')
```

Además, ten encuenta lo que devuelve cada operación que aplicas. **filter** devuelve un DataFrame filtrado, pero **show** no devuelve nada, así que si asignas el resultado de **show** a tu variable, la variable valdrá **None**.
```
df = df.filter(F.col('degree') == 'Engineering').show()  # Ahora df = None
```

In [10]:
df.show()

+----+----------------+---+------------+-----+
|  id|            name|age|      degree|grade|
+----+----------------+---+------------+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|
|1002|      Luis Gomez| 22| Engineering|  9.0|
|1003|     Maria Lopez| 21| Engineering|  7.8|
|1004|  Javier Sanchez| 23|Architecture|  6.5|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|
+----+----------------+---+------------+-----+



# Create and drop columns

In [11]:
df = df.withColumn('stars', F.col('grade')/2)
df.show()

+----+----------------+---+------------+-----+------------------+
|  id|            name|age|      degree|grade|             stars|
+----+----------------+---+------------+-----+------------------+
|1001|       Ana Perez| 20| Engineering|  8.5|              4.25|
|1002|      Luis Gomez| 22| Engineering|  9.0|               4.5|
|1003|     Maria Lopez| 21| Engineering|  7.8|3.9000000953674316|
|1004|  Javier Sanchez| 23|Architecture|  6.5|              3.25|
|1005|Carmen Rodriguez| 20|Architecture|  8.2| 4.099999904632568|
+----+----------------+---+------------+-----+------------------+



Se pueden concatenar operaciones y hacer referencia a columnas creadas en operaciones previas.

In [12]:
df = df.withColumn('stars', F.col('grade') / 2)\
  .withColumn('stars', F.round(F.col('stars'), 1))

df.show()

+----+----------------+---+------------+-----+-----+
|  id|            name|age|      degree|grade|stars|
+----+----------------+---+------------+-----+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.3|
|1002|      Luis Gomez| 22| Engineering|  9.0|  4.5|
|1003|     Maria Lopez| 21| Engineering|  7.8|  3.9|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.3|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.1|
+----+----------------+---+------------+-----+-----+



Se pueden anidar funciones unas dentro de otras

In [13]:
df = df.withColumn('stars', F.round(F.col('grade') / 2, 1))

df.show()

+----+----------------+---+------------+-----+-----+
|  id|            name|age|      degree|grade|stars|
+----+----------------+---+------------+-----+-----+
|1001|       Ana Perez| 20| Engineering|  8.5|  4.3|
|1002|      Luis Gomez| 22| Engineering|  9.0|  4.5|
|1003|     Maria Lopez| 21| Engineering|  7.8|  3.9|
|1004|  Javier Sanchez| 23|Architecture|  6.5|  3.3|
|1005|Carmen Rodriguez| 20|Architecture|  8.2|  4.1|
+----+----------------+---+------------+-----+-----+



In [14]:
df = df.withColumn('first_name', F.split(F.col('name'), ' ')[0]) \
  .withColumn('last_name', F.split(F.col('name'), ' ')[1]) \
  .drop('name')

df.show()

+----+---+------------+-----+-----+----------+---------+
|  id|age|      degree|grade|stars|first_name|last_name|
+----+---+------------+-----+-----+----------+---------+
|1001| 20| Engineering|  8.5|  4.3|       Ana|    Perez|
|1002| 22| Engineering|  9.0|  4.5|      Luis|    Gomez|
|1003| 21| Engineering|  7.8|  3.9|     Maria|    Lopez|
|1004| 23|Architecture|  6.5|  3.3|    Javier|  Sanchez|
|1005| 20|Architecture|  8.2|  4.1|    Carmen|Rodriguez|
+----+---+------------+-----+-----+----------+---------+



# Group by

**groupBy** también es una transformación que se ejecuta de forma **lazy**

In [15]:
df.groupBy('degree').agg(F.avg(F.col('stars').alias('average_stars')))

DataFrame[degree: string, avg(stars AS average_stars): double]

In [16]:
df.groupBy('degree').agg(F.avg(F.col('stars').alias('average_stars'))).show()

+------------+---------------------------+
|      degree|avg(stars AS average_stars)|
+------------+---------------------------+
| Engineering|          4.233333333333333|
|Architecture|         3.6999999999999997|
+------------+---------------------------+



Se pueden aplicar varios cálculos de agregación

In [17]:
df.groupBy('degree').agg(
    F.avg(F.col('stars')).alias('average_stars'),
    F.max(F.col('stars')).alias('max_stars'),
    F.min(F.col('stars')).alias('min_stars'),
).show()

+------------+------------------+---------+---------+
|      degree|     average_stars|max_stars|min_stars|
+------------+------------------+---------+---------+
| Engineering| 4.233333333333333|      4.5|      3.9|
|Architecture|3.6999999999999997|      4.1|      3.3|
+------------+------------------+---------+---------+



# Join

In [18]:
data = [
    (1001, "Huelva"),
    (1002, "Sevilla"),
    (1003, "Huelva"),
    (1004, "Sevilla"),
    (1005, "Huelva")
]

schema = T.StructType([
    T.StructField("id", T.IntegerType(), True),
    T.StructField("city", T.StringType(), True)
])

df_city = spark.createDataFrame(data, schema)

df_city.show()

+----+-------+
|  id|   city|
+----+-------+
|1001| Huelva|
|1002|Sevilla|
|1003| Huelva|
|1004|Sevilla|
|1005| Huelva|
+----+-------+



In [19]:
df = df.join(df_city, on='id', how='inner')
df.show()

+----+---+------------+-----+-----+----------+---------+-------+
|  id|age|      degree|grade|stars|first_name|last_name|   city|
+----+---+------------+-----+-----+----------+---------+-------+
|1001| 20| Engineering|  8.5|  4.3|       Ana|    Perez| Huelva|
|1002| 22| Engineering|  9.0|  4.5|      Luis|    Gomez|Sevilla|
|1003| 21| Engineering|  7.8|  3.9|     Maria|    Lopez| Huelva|
|1004| 23|Architecture|  6.5|  3.3|    Javier|  Sanchez|Sevilla|
|1005| 20|Architecture|  8.2|  4.1|    Carmen|Rodriguez| Huelva|
+----+---+------------+-----+-----+----------+---------+-------+



# Write

El resultado de esta escritura será una carpeta llamada **students.csv** que contendrá partes de un csv

In [20]:
df.write.csv('/content/data/csv/students.csv')

En Spark se indican las rutas para leer o escribir se indican a nivel de **carpeta**, no de fichero

In [21]:
df.write.csv('/content/data/csv/students/')

Automáticamente, Spark escribe la cantidad de ficheros que considere óptima. El equilibrio entre cantidad de **ficheros** y **tamaño de fichero**, es algo crucial en problemas de Big Data para conseguir un **rendimiento**.

In [22]:
df.repartition(5).write.csv('/content/data/csv/students_repartition/')

El particionado es otro aspecto crítico en problemas de Big Data. Permite filtrar de forma más eficiente un DataFrame, permitiendo cargar únicamente la partición de interés. Si bien, también se debe **balancear el número de particiones y la profundidad de las particiones** para tener un buen rendimiento.

Para ello, se debe saber de antemano **cómo se va a consumir la información** y en base a ello, seleccionar las columnas con las que particionar.

In [23]:
df.write.partitionBy('city').csv('/content/data/csv/students_partitioned/')

El formato de fichero habitual para trabajar con Spark es **parquet**. Es un formato **binario** optimizado para Spark que permite comprimir la información hasta más de un 90% y leer partes del fichero sin tener que cargarlo al completo.

In [24]:
df.write.partitionBy('city').parquet('/content/data/parquet/students/')

Se puede particionar por más de una columna indicándola en el **partitionBy**

In [27]:
df.write.partitionBy('city', 'degree').parquet('/content/data/parquet/students_v2/')

# Read

Observa que al haber leído a nivel de partición, la tabla no tiene la columna **city**. Esto se debe a que la **columna de partición no se guarda en el fichero** porque se puede deducir de la partición. Así, se ahorra espacio en disco.

In [25]:
df = spark.read.load('/content/data/parquet/students/city=Huelva/')

df.show()

+----+---+------------+-----+-----+----------+---------+
|  id|age|      degree|grade|stars|first_name|last_name|
+----+---+------------+-----+-----+----------+---------+
|1001| 20| Engineering|  8.5|  4.3|       Ana|    Perez|
|1003| 21| Engineering|  7.8|  3.9|     Maria|    Lopez|
|1005| 20|Architecture|  8.2|  4.1|    Carmen|Rodriguez|
+----+---+------------+-----+-----+----------+---------+



Al indicar la ruta raíz (**students**), Spark lee todos las particiones y ficheros y los concatena automáticamente en una sola tabla. De esta forma el desarrollador se despreocupa de concatenar manualmente cada fichero y cada partición.

In [26]:
df = spark.read.load('/content/data/parquet/students/')

df.show()

+----+---+------------+-----+-----+----------+---------+-------+
|  id|age|      degree|grade|stars|first_name|last_name|   city|
+----+---+------------+-----+-----+----------+---------+-------+
|1001| 20| Engineering|  8.5|  4.3|       Ana|    Perez| Huelva|
|1003| 21| Engineering|  7.8|  3.9|     Maria|    Lopez| Huelva|
|1005| 20|Architecture|  8.2|  4.1|    Carmen|Rodriguez| Huelva|
|1002| 22| Engineering|  9.0|  4.5|      Luis|    Gomez|Sevilla|
|1004| 23|Architecture|  6.5|  3.3|    Javier|  Sanchez|Sevilla|
+----+---+------------+-----+-----+----------+---------+-------+

